In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd /home/optima/mhaderer/AILS-MICCAI-UWF4DR-Challenge

[Errno 2] No such file or directory: '/home/optima/mhaderer/AILS-MICCAI-UWF4DR-Challenge'
/Users/moritz/Documents/Master/AILS-MICCAI-UWF4DR-Challenge/notebooks


/Users/moritz/miniconda3/envs/AILS-MICCAI-UWF4DR-Challenge/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [3]:
# setup
#!apt-get update
#!pip install python-dotenv
#!pip install loguru
#!pip install gdown
#!pip install typer

In [4]:
# load data and unzip data

#!python tools/download_data_and_chkpts.py

In [10]:
# imports

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import wandb

from sklearn.metrics import roc_auc_score, average_precision_score

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import time

from ails_miccai_uwf4dr_challenge.models.metrics import sensitivity_score, specificity_score
from ails_miccai_uwf4dr_challenge.dataset import ChallengeTaskType, CustomDataset, DatasetBuilder, DatasetOriginationType
from ails_miccai_uwf4dr_challenge.preprocess_augmentations import ResidualGaussBlur, MultiplyMask
from ails_miccai_uwf4dr_challenge.models.trainer import DefaultMetricsEvaluationStrategy, Metric, MetricCalculatedHook, \
    NumBatches, Trainer, EpochTrainingStrategy, EpochValidationStrategy, DefaultEpochTrainingStrategy, \
    DefaultBatchTrainingStrategy, TrainingContext, PersistBestModelOnEpochEndHook
from ails_miccai_uwf4dr_challenge.models.architectures.task1_automorph_plain import AutoMorphModel
from ails_miccai_uwf4dr_challenge.models.architectures.task1_efficientnet_plain import Task1EfficientNetB4
from ails_miccai_uwf4dr_challenge.models.architectures.task2_efficientnetb0_plain import Task2EfficientNetB0 
from ails_miccai_uwf4dr_challenge.config import WANDB_API_KEY, PROJ_ROOT

wandb.login(key=WANDB_API_KEY)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/moritz/.netrc


True

In [11]:
# select device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: " + str(device))

Device: cpu


## Train automorph model

In [7]:



preprocessing = A.Compose([
        A.Resize(800, 1016, p=1),
        A.Normalize(mean=[0.406, 0.485, 0.456], std=[0.225, 0.229, 0.224], p=1),
        #ResidualGaussBlur(p=1),
        MultiplyMask(p=1),
        A.Resize(800, 1016, p=1),
        #A.Equalize(p=0.1),
        #A.CLAHE(clip_limit=5., p=0.3)
    ])

augment_train = A.Compose([
        #A.VerticalFlip(p=0.5),
        #A.HorizontalFlip(p=0.5),
        #A.Affine(rotate=15, rotate_method='ellipse', p=0.5),
        ToTensorV2(p=1)
    ])

augment_val = A.Compose([
        ToTensorV2(p=1)
    ])


transforms_train = A.Compose([
    preprocessing,
    augment_train
])

transforms_val = A.Compose([
    preprocessing,
    augment_val
])

In [12]:
def train_automorph(config=None):

    wandb.init(project="task2", config=config)
    config = wandb.config

    dataset = DatasetBuilder(dataset=DatasetOriginationType.ORIGINAL, task=ChallengeTaskType.TASK2)
    train_data, val_data = dataset.get_train_val()

    train_dataset = CustomDataset(train_data, transform=transforms_train)
    val_dataset = CustomDataset(val_data, transform=transforms_val)

    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu" if torch.backends.mps.is_available() else "cpu") #don't use mps, it takes ages, whyever that is the case!?!
    print(f"Using device: {device}")

    try:
        model = AutoMorphModel(enc_frozen=True)
    except ValueError:
        raise ValueError(f"Unknown model: AutoMorphModel")

    model.to(device)
    
    print("Training model: ", model.__class__.__name__)    

    metrics = [
        Metric('auroc', roc_auc_score),
        Metric('auprc', average_precision_score),
        Metric('accuracy', lambda y_true, y_pred: (y_pred.round() == y_true).mean()),
        Metric('sensitivity', sensitivity_score),
        Metric('specificity', specificity_score)
    ]

    class WandbLoggingHook(MetricCalculatedHook):
        def on_metric_calculated(self, training_context: TrainingContext, metric: Metric, result, last_metric_for_epoch: bool):
            import wandb
            wandb.log(data={metric.name: result}, commit=last_metric_for_epoch)

    metrics_eval_strategy = DefaultMetricsEvaluationStrategy(metrics).register_metric_calculated_hook(WandbLoggingHook())

    def combined_losses(pred, target):
        bce = F.binary_cross_entropy_with_logits(pred, target) * 0.5
        smooth_l1 = F.smooth_l1_loss(pred, target) * 0.5
        return bce + smooth_l1

    criterion = combined_losses
    optimizer = optim.AdamW(model.parameters(), lr=config["learning_rate"])
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

    trainer = Trainer(model, train_loader, val_loader, criterion, optimizer, lr_scheduler, device, 
                        metrics_eval_strategy=metrics_eval_strategy)

    # build a file name for the model weights containing current timestamp and the model class
    training_timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")
    persist_model_hook = PersistBestModelOnEpochEndHook(f"best_model_{training_timestamp}.pth")
    trainer.add_epoch_end_hook(persist_model_hook) # TODO uncomment this line to save the best model

    #print("First train 2 epochs 2 batches to check if everything works - you can comment these two lines after the code has stabilized...")
    #trainer.train(num_epochs=2, num_batches=NumBatches.TWO_FOR_INITIAL_TESTING)
    
    print("Now train train train")
    trainer.train(num_epochs=config.epochs)
    wandb.finish()
    print("Finished training")

In [13]:


config = {
    "dataset": "original",
    "task": "task2",
    "augmentations": "resize only",
    "loss_func": "combined losses",
    "learning_rate": 1e-3,
    "epochs": 50,
    "batch_size": 32,
    "model_type": "AutoMorphModel encoder unfrozen"
}

WANDB_HTTP_TIMEOUT=300
WANDB_INIT_TIMEOUT =600
WANDB_DEBUG=True


In [14]:
#train_automorph(config)

## Train efficientnet b0

In [15]:
transforms_train = A.Compose([
        A.Resize(800, 1016, p=1),
        MultiplyMask(p=1),
        #ResidualGaussBlur(p=.5),
        #A.Equalize(p=0.3),
        #A.CLAHE(clip_limit=5., p=0.5)
        #A.VerticalFlip(p=0.3),
        #A.HorizontalFlip(p=0.3),
        #A.Affine(rotate=15, rotate_method='ellipse', p=0.3),
        A.Normalize(mean=[0.406, 0.485, 0.456], std=[0.225, 0.229, 0.224], p=1),
        ToTensorV2(p=1)
    ])

transforms_val = A.Compose([
        A.Resize(800, 1016, p=1),
        MultiplyMask(p=1),
        A.Normalize(mean=[0.406, 0.485, 0.456], std=[0.225, 0.229, 0.224], p=1),
        ToTensorV2(p=1)
    ])



In [18]:
def train_efficientnetb0(config=None):

    wandb.init(project="task2", config=config)
    config = wandb.config

    dataset = DatasetBuilder(dataset=DatasetOriginationType.ORIGINAL, task=ChallengeTaskType.TASK2)
    train_data, val_data = dataset.get_train_val()

    train_dataset = CustomDataset(train_data, transform=transforms_train)
    val_dataset = CustomDataset(val_data, transform=transforms_val)

    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu" if torch.backends.mps.is_available() else "cpu") #don't use mps, it takes ages, whyever that is the case!?!
    print(f"Using device: {device}")

    try:
        model = Task2EfficientNetB0(num_classes=1)
    except ValueError:
        raise ValueError(f"Unknown model: Task2EfficientNetB0")

    model.to(device)
    
    print("Training model: ", model.__class__.__name__)    

    metrics = [
        Metric('auroc', roc_auc_score),
        Metric('auprc', average_precision_score),
        Metric('accuracy', lambda y_true, y_pred: (y_pred.round() == y_true).mean()),
        Metric('sensitivity', sensitivity_score),
        Metric('specificity', specificity_score)
    ]

    class WandbLoggingHook(MetricCalculatedHook):
        def on_metric_calculated(self, training_context: TrainingContext, metric: Metric, result, last_metric_for_epoch: bool):
            import wandb
            wandb.log(data={metric.name: result}, commit=last_metric_for_epoch)

    metrics_eval_strategy = DefaultMetricsEvaluationStrategy(metrics).register_metric_calculated_hook(WandbLoggingHook())

    def combined_losses(pred, target):
        bce = F.binary_cross_entropy_with_logits(pred, target) * 0.5
        smooth_l1 = F.smooth_l1_loss(pred, target) * 0.5
        return bce + smooth_l1

    criterion = combined_losses
    optimizer = optim.AdamW(model.parameters(), lr=config["learning_rate"])
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

    trainer = Trainer(model, train_loader, val_loader, criterion, optimizer, lr_scheduler, device, 
                        metrics_eval_strategy=metrics_eval_strategy)

    # build a file name for the model weights containing current timestamp and the model class
    training_timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")
    persist_model_hook = PersistBestModelOnEpochEndHook(f"best_model_{training_timestamp}.pth")
    trainer.add_epoch_end_hook(persist_model_hook) # TODO uncomment this line to save the best model

    #print("First train 2 epochs 2 batches to check if everything works - you can comment these two lines after the code has stabilized...")
    #trainer.train(num_epochs=2, num_batches=NumBatches.TWO_FOR_INITIAL_TESTING)
    
    print("Now train train train")
    trainer.train(num_epochs=config.epochs)
    wandb.finish()
    print("Finished training")

In [ ]:


config = {
    "dataset": "original",
    "task": "task2",
    "augmentations": "resize only",
    "loss_func": "combined losses",
    "learning_rate": 1e-3,
    "epochs": 20,
    "batch_size": 16,
    "model_type": "Task2EfficientNetB0"
}

WANDB_HTTP_TIMEOUT=300
WANDB_INIT_TIMEOUT =600
WANDB_DEBUG=True


In [17]:
train_efficientnetb0(config)